In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("setup").getOrCreate()

In [2]:
#read_files
accounts = spark.read.options(header='True', inferSchema='True', delimiter=';') \
     .csv("/home/jovyan/accounts.csv")
country_abbreviation = spark.read.options(header='True', inferSchema='True', delimiter=';') \
     .csv("/home/jovyan/country_abbreviation.csv")
transactions = spark.read.options(header='True', inferSchema='True', delimiter=';') \
     .csv("/home/jovyan/transactions.csv")

In [20]:
swiss_abb = country_abbreviation.select("abbreviation").filter(country_abbreviation.country_full_name == "Switzerland").collect()[0].__getitem__('abbreviation')

In [34]:
accounts = accounts.alias('acc')
transactions = transactions.alias('tr')
swiss_earnings = accounts.filter(accounts.country == swiss_abb).join(transactions.filter(transactions.amount > 0),accounts.id == transactions.id, "left").select('acc.first_name', 'acc.last_name', 'tr.transaction_date', 'tr.amount')

+----------+---------+----------------+-------+
|first_name|last_name|transaction_date| amount|
+----------+---------+----------------+-------+
| Frederick| Morrison|      2021-06-16|  512.3|
| Frederick| Morrison|      2012-10-24|8707.01|
| Frederick| Morrison|      2020-09-04|9360.72|
| Frederick| Morrison|      2019-05-10|7892.81|
| Frederick| Morrison|      2016-06-25|1582.28|
| Frederick| Morrison|      2016-05-14|5574.68|
| Frederick| Morrison|      2021-01-11|9686.86|
|Maximilian|  Chapman|      2016-05-18|6182.78|
|Maximilian|  Chapman|      2016-03-31|8899.88|
|Maximilian|  Chapman|      2013-10-12|2047.12|
|Maximilian|  Chapman|      2016-10-17|2502.55|
|Maximilian|  Chapman|      2020-02-16|9933.39|
|Maximilian|  Chapman|      2012-04-13|3725.31|
|Maximilian|  Chapman|      2019-06-12|6172.79|
|Maximilian|  Chapman|      2016-07-05|7405.78|
|Maximilian|  Chapman|      2016-05-11|7547.69|
|   Chester|Henderson|      2013-07-10|1318.64|
|   Chester|Henderson|      2015-11-02|9

In [39]:
from pyspark.sql.functions import year, concat, col, lit

swiss_earnings_filtered = swiss_earnings.select(concat('first_name',lit(' '),'last_name').alias("full_name"),year("transaction_date").alias('year'),"amount")
swiss_earnings_agg = swiss_earnings_filtered.groupBy("full_name").pivot("year").sum("amount").show()

+----------------+--------+------------------+--------+--------+--------+------------------+-------+-------+--------+------------------+------------------+
|       full_name|    2011|              2012|    2013|    2014|    2015|              2016|   2017|   2018|    2019|              2020|              2021|
+----------------+--------+------------------+--------+--------+--------+------------------+-------+-------+--------+------------------+------------------+
|  Adison Douglas|    null|          10622.48|14322.94|    null|    null|              null|6328.92|8248.97|    null|              null|           3406.01|
|   Darcy Edwards|    null|              null|    null|16848.72|    null|           7922.11|10337.6|   null| 3722.62|            7545.8|              null|
|  Olivia Hawkins| 8112.16|           1196.61| 7345.82|  576.65|    null|             82.74|   null|   null| 1244.63|              null|           4903.52|
|   Connie Gibson|    null|           8486.84| 3664.69| 7385.69|